In [ ]:
from IPython.display import clear_output

!pip install transformers accelerate
!pip install datasets
!pip install wandb
# !pip install nlpaug
# !pip install sacremoses
# !pip install sentencepiece
# !pip install langdetect

clear_output()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import cuda
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    BertModel,
    DataCollatorWithPadding,
    BertForSequenceClassification,
    BertForTokenClassification,
    AutoModelForMaskedLM,
    BertTokenizer,
    AutoModelForTokenClassification,
    pipeline,
    TrainingArguments,
    Trainer,
    BertConfig,
    EarlyStoppingCallback
)

from sklearn.model_selection import train_test_split
from google.colab import drive, output
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
import locale
from torch.nn import BCELoss, BCEWithLogitsLoss
# import wandb
import random

# wandb.login()

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

drive.mount('/content/drive', force_remount=True)
clear_output()

In [ ]:
WORK_FOLDER = 'drive/MyDrive/ML/GPT or Human'
DEVICE_NUM = 0
DEVICE = f"cuda:{DEVICE_NUM}" if cuda.is_available() else "cpu"
BATCH_SIZE = 16
EPOCHS=10

In [ ]:
df = pd.read_csv(f'{WORK_FOLDER}/data/train.csv')

In [ ]:
df.drop(columns=['q_id', 'line_id'], inplace=True)
df['label'] = df['label'].map({'ai_answer': 1, 'hu_answer': 0}).values
df.head()

,q_title,label,ans_text
0,"Какие комплектующие должны быть в компьютере, ...",0,"Да ничего особенного. :)\nКорпус должен быть, ..."
1,"Какие комплектующие должны быть в компьютере, ...",1,Здравствуйте! Спасибо за интересный вопрос. Дл...
2,Loading a Reusable UITableViewCell from a Nib,1,"To load a reusable UITableViewCell from a Nib,..."
3,Loading a Reusable UITableViewCell from a Nib,0,"Actually, since you are building the cell in I..."
4,How can I change UIButton title color?,0,You can use -[UIButton setTitleColor:forState:...


In [ ]:
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
multi_lan_model = "bert-base-multilingual-cased"
multi_tokenizer = BertTokenizer.from_pretrained(multi_lan_model)
clear_output()

In [ ]:
# tokenized_train = multi_tokenizer(
#     train_df["ans_text"].tolist(),
#     padding=True,
#     truncation=True,
#     return_tensors="pt"
# )
# train_input_ids, train_masks = (
#     tokenized_train["input_ids"],
#     tokenized_train["attention_mask"],
# )
# train_labels = torch.tensor(train_df["label"].values).unsqueeze(-1)

tokenized_valid = multi_tokenizer(
    valid_df["ans_text"].tolist(),
    padding=True,
    truncation=True,
    return_tensors="pt",
)
# valid_input_ids, valid_masks = (
#     tokenized_valid["input_ids"],
#     tokenized_valid["attention_mask"],
# )
# valid_labels = torch.tensor(valid_df["label"].values).unsqueeze(-1)

# # train_dataset = TensorDataset(train_input_ids, train_masks, train_labels)
# val_dataset = TensorDataset(valid_input_ids, valid_masks, valid_labels)

In [ ]:
tokenized_valid

{'input_ids': tensor([[   101,  11220,  10454,  ...,    121,    119,    102],
        [   101,    107, 105104,  ...,      0,      0,      0],
        [   101,    523,    117,  ...,      0,      0,      0],
        ...,
        [   101,  69345,  15597,  ...,  10297,  12709,    102],
        [   101,  11469,  13708,  ...,      0,      0,      0],
        [   101,    516,  35865,  ...,      0,      0,      0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

## *Бейзлайн модель*

In [ ]:
# base_model = BertForSequenceClassification.from_pretrained(
#     multi_lan_model,
#     num_labels = 2,
#     output_attentions = False,
#     output_hidden_states = False
# )

# base_model.to(DEVICE)
# clear_output()

In [ ]:
class MyDataCollator:
    def __call__(self, batch):
        return {
            "input_ids": torch.stack([t[0] for t in batch]),
            "attention_mask": torch.stack([t[1] for t in batch]),
            "labels": torch.stack([t[2] for t in batch]),
        }

def f1_metric(preds):
    y_true = preds.label_ids
    y_pred = preds.predictions.argmax(-1)
    return {"F1": f1_score(y_true, y_pred)}


training_args = TrainingArguments(
    output_dir=f"{WORK_FOLDER}/Runs/Base",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_dir=f"{WORK_FOLDER}/Runs",
    remove_unused_columns=False,
    include_inputs_for_metrics=True,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="F1",
    greater_is_better=True,
)

In [ ]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=MyDataCollator(),
    compute_metrics=f1_metric
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,F1
500,0.162800,0.192270,0.955340
1000,0.026000,0.163972,0.976143


Step,Training Loss,Validation Loss,F1
500,0.162800,0.192270,0.955340
1000,0.026000,0.163972,0.976143
1500,0.000000,0.131455,0.983017
2000,0.000000,0.141592,0.983017


TrainOutput(global_step=2410, training_loss=0.03918642372654218, metrics={'train_runtime': 3817.0127, 'train_samples_per_second': 10.1, 'train_steps_per_second': 0.631, 'total_flos': 1.0142931184128e+16, 'train_loss': 0.03918642372654218, 'epoch': 10.0})

#### *Другие гиперпараметры у базовой модели*

In [ ]:
training_args.output_dir = f"{WORK_FOLDER}/Runs/Base2"
training_args.num_train_epochs = 15
training_args.weight_decay = 0.01
training_args.learning_rate = 1e-6

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=MyDataCollator(),
    compute_metrics=f1_metric,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

*Скор упал*

##*Другой классификатор*

In [ ]:
class TwoLayerMLPClassifier(nn.Module):
    def __init__(self, hidden_size, num_classes, dropout_rate=0.1):
        super().__init__()
        self.layer1 = nn.Linear(hidden_size, hidden_size)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.layer2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.layer2(x)
        return x

In [ ]:
base_model.classifier = TwoLayerMLPClassifier(base_model.config.hidden_size, 2)

training_args.output_dir = f"{WORK_FOLDER}/Runs/Class2"
training_args.num_train_epochs = 15
training_args.weight_decay = 0.01
training_args.learning_rate = 1e-6

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=MyDataCollator(),
    compute_metrics=f1_metric,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,F1
500,0.164400,0.079431,0.981057
1000,0.016600,0.061808,0.984985
1500,0.007300,0.070297,0.984000


Step,Training Loss,Validation Loss,F1
500,0.164400,0.079431,0.981057
1000,0.016600,0.061808,0.984985
1500,0.007300,0.070297,0.984000
2000,0.004500,0.078709,0.984000


TrainOutput(global_step=2000, training_loss=0.04819664597511292, metrics={'train_runtime': 3109.9205, 'train_samples_per_second': 18.594, 'train_steps_per_second': 1.162, 'total_flos': 8475491924656128.0, 'train_loss': 0.04819664597511292, 'epoch': 8.3})

*^ Это пока лучшая модель*

## *Дисбаланс классов + бейзлайн / другой классификатор*

In [ ]:
train_df['label'].value_counts()

1    1932
0    1923
Name: label, dtype: int64

In [ ]:
class_count_1, class_count_0 = train_df['label'].value_counts()

class_0 = train_df[train_df['label'] == 0]
class_1 = train_df[train_df['label'] == 1]

class_0_over = class_0.sample(class_count_1, replace=True)

train_df_balanced = pd.concat([
    class_0_over, class_1
    ], axis=0).sample(frac=1)

In [ ]:
train_df_balanced['label'].value_counts()

0    1932
1    1932
Name: label, dtype: int64

In [ ]:
tokenized_train = multi_tokenizer(
    train_df_balanced["ans_text"].tolist(),
    padding=True,
    truncation=True,
    return_tensors="pt"
)
train_input_ids, train_masks = (
    tokenized_train["input_ids"],
    tokenized_train["attention_mask"],
)
train_labels = torch.tensor(train_df_balanced["label"].values).unsqueeze(-1)
train_dataset = TensorDataset(train_input_ids, train_masks, train_labels)

In [ ]:
base_model = BertForSequenceClassification.from_pretrained(
    multi_lan_model,
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False
)

base_model.to(DEVICE)
clear_output()

training_args.output_dir = f"{WORK_FOLDER}/Runs/Base4"
training_args.num_train_epochs = 30
training_args.weight_decay = 0.01
training_args.learning_rate = 1e-6

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=MyDataCollator(),
    compute_metrics=f1_metric,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],

)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,F1
500,0.407700,0.333023,0.893773


In [ ]:
trainer.train(f"{WORK_FOLDER}/Runs/Base2/checkpoint-2000")

Step,Training Loss,Validation Loss


TrainOutput(global_step=2410, training_loss=3.891109384565433e-06, metrics={'train_runtime': 634.6782, 'train_samples_per_second': 60.739, 'train_steps_per_second': 3.797, 'total_flos': 1.0142931184128e+16, 'train_loss': 3.891109384565433e-06, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.1908539980649948,
 'eval_F1': 0.9752229930624381,
 'eval_runtime': 32.6148,
 'eval_samples_per_second': 29.557,
 'eval_steps_per_second': 1.87,
 'epoch': 20.0}

In [ ]:
trainer.save_model(f"{WORK_FOLDER}/Runs/Base2/maybe_better_cls_balanced")

In [ ]:
# for param in base_model.base_model.parameters():
#     param.requires_grad = False

base_model.classifier = TwoLayerMLPClassifier(base_model.config.hidden_size, 2)

training_args.output_dir = f"{WORK_FOLDER}/Runs/Class4"
training_args.num_train_epochs = 30
training_args.weight_decay = 0.01
training_args.learning_rate = 1e-6

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=MyDataCollator(),
    compute_metrics=f1_metric,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: lisabeth-shevtsova (liza-i-pivko). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1
500,0.607400,0.405490,0.875000
1000,0.354800,0.251672,0.920152
1500,0.183900,0.213309,0.939013
2000,0.098300,0.174511,0.952102
2500,0.058700,0.219478,0.951267
3000,0.045700,0.246394,0.948693


Step,Training Loss,Validation Loss,F1
500,0.607400,0.405490,0.875000
1000,0.354800,0.251672,0.920152
1500,0.183900,0.213309,0.939013
2000,0.098300,0.174511,0.952102
2500,0.058700,0.219478,0.951267
3000,0.045700,0.246394,0.948693


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 21>:21                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1664 in train                    │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1940 in _inner_training_loop     │
│                                                                                                  │
│   1937 │   │   │   │   │   with model.no_sync():                                                 │
│   1938 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1939 │   │   │   │   else:                                                                     │
│ ❱ 1940 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1941 │   │   │   │                                                                             │
│   1942 │   │   │   │   if (                                                                      │
│   1943 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2753 in training_step            │
│                                                                                                  │
│   2750 │   │   │   # loss gets scaled under gradient_accumulation_steps in deepspeed             │
│   2751 │   │   │   loss = self.deepspeed.backward(loss)                                          │
│   2752 │   │   else:                                                                             │
│ ❱ 2753 │   │   │   loss.backward()                                                               │
│   2754 │   │                                                                                     │
│   2755 │   │   return loss.detach()                                                              │
│   2756                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/_tensor.py:487 in backward                         │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│ ❱  487 │   │   torch.autograd.backward(                                                          │
│    488 │   │   │   self, gradient, retain_graph, create_gra

In [ ]:
# play notification :D
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

## *Языки*

In [ ]:
from langdetect import detect

In [ ]:
train_langs = train_df_balanced['ans_text'].apply(detect)
train_langs.value_counts()

en    2170
ru    1691
ca       3
Name: ans_text, dtype: int64

In [ ]:
train_df_balanced['lang'] = train_langs
train_df_balanced['lang'] = train_df_balanced['lang'].apply(lambda l: 'en' if l != 'ru' else 'ru')

train_df_balanced.head()

,q_title,label,ans_text,lang
2246,Can multithreading be implemented on a single ...,1,"Yes, multithreading can be implemented on a si...",en
1498,How do I get the intersection between two arra...,1,"Well, well, well, looks like someone needs to ...",en
2880,"Есть ли что-то из классической литературы, что...",0,Примерно раз в 5 лет перечитываю всего Оскара ...,ru
1031,iOS app error - Can't add self as subview,0,I am speculating based on something similar th...,en
1313,Какую книгу для подростков 14-15 лет вы считае...,0,"Помню как (правда, это было уже на пороге 16 л...",ru


In [ ]:
train_df_balanced.lang.unique()

array(['en', 'ru'], dtype=object)

## *Augmentation*

In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [ ]:
back_translation_aug_ru = naw.BackTranslationAug(
    from_model_name='Helsinki-NLP/opus-mt-ru-en',
    to_model_name='Helsinki-NLP/opus-mt-en-ru',
    device=DEVICE,
    batch_size=8
)

clear_output()

class_rus = train_df_balanced[train_df_balanced.lang == 'ru'].copy()
aug_rus = back_translation_aug_ru.augment(class_rus.ans_text.to_list())

In [ ]:
# play notification :D
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
back_translation_aug_en = naw.BackTranslationAug(
    from_model_name='Helsinki-NLP/opus-mt-en-ru',
    to_model_name='Helsinki-NLP/opus-mt-ru-en',
    device=DEVICE,
    batch_size=8
)
class_eng = train_df_balanced[train_df_balanced.lang == 'en'].copy()
aug_eng = back_translation_aug_en.augment(class_eng.ans_text.to_list())

In [ ]:
torch.cuda.empty_cache()

In [ ]:
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
class_rus['ans_text'] = aug_rus
class_eng['ans_text'] = aug_eng

In [ ]:
train_df_balanced_aug = pd.concat([train_df_balanced, class_rus, class_eng]).sample(frac=1).reset_index(drop=True)
train_df_balanced_aug.to_csv(f"{WORK_FOLDER}/train_df_balanced_aug.csv")

In [ ]:
train_df_balanced_aug = pd.read_csv(f"{WORK_FOLDER}/train_df_balanced_aug.csv")

In [ ]:
tokenized_train = multi_tokenizer(
    train_df_balanced_aug["ans_text"].tolist(),
    padding=True,
    truncation=True,
    return_tensors="pt"
)
train_input_ids, train_masks = (
    tokenized_train["input_ids"],
    tokenized_train["attention_mask"],
)
train_labels = torch.tensor(train_df_balanced_aug["label"].values).unsqueeze(-1)
train_dataset = TensorDataset(train_input_ids, train_masks, train_labels)

base_model = BertForSequenceClassification.from_pretrained(
    multi_lan_model,
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False
)

base_model.to(DEVICE)
clear_output()

training_args.output_dir = f"{WORK_FOLDER}/Runs/Base4"
training_args.num_train_epochs = 10

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=MyDataCollator(),
    compute_metrics=f1_metric,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,F1
500,0.245300,0.125210,0.964567
1000,0.057100,0.252380,0.960938
1500,0.027100,0.136579,0.974155
2000,0.014500,0.190755,0.973081
2500,0.014000,0.276504,0.967552


Step,Training Loss,Validation Loss,F1
500,0.245300,0.125210,0.964567
1000,0.057100,0.252380,0.960938
1500,0.027100,0.136579,0.974155
2000,0.014500,0.190755,0.973081
2500,0.014000,0.276504,0.967552
3000,0.002400,0.298018,0.966535


TrainOutput(global_step=3000, training_loss=0.06007338134447734, metrics={'train_runtime': 4884.3315, 'train_samples_per_second': 15.822, 'train_steps_per_second': 0.989, 'total_flos': 1.262933065728e+16, 'train_loss': 0.06007338134447734, 'epoch': 6.21})

In [ ]:
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

## *SCRIPT FOR INFERENCE*

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import TensorDataset, DataLoader


class TwoLayerMLPClassifier(nn.Module):
    def __init__(self, hidden_size, num_classes, dropout_rate=0.1):
        super().__init__()
        self.layer1 = nn.Linear(hidden_size, hidden_size)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.layer2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.layer2(x)
        return x


class CustomBertForSequenceClassification(BertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.classifier = TwoLayerMLPClassifier(
            self.bert.config.hidden_size, self.num_labels)


MULTI_LANG_BERT = "bert-base-multilingual-cased"

if __name__ == "__main__":
    df_test = pd.read_csv(f"{WORK_FOLDER}/data/train.csv")
    multi_tokenizer = BertTokenizer.from_pretrained(
        MULTI_LANG_BERT, model_max_length=512)
    tokenized_test = multi_tokenizer(
        df_test["ans_text"].tolist(),
        padding=True,
        truncation=True,
        return_tensors="pt",
    )
    test_input_ids, test_masks = (
        tokenized_test["input_ids"],
        tokenized_test["attention_mask"],
    )
    test_dataset = TensorDataset(test_input_ids, test_masks)
    loader = DataLoader(test_dataset, batch_size=256)

    model = CustomBertForSequenceClassification.from_pretrained(
        f"{WORK_FOLDER}/Runs/Class2/checkpoint-1000",
        num_labels=2,
        output_attentions=False,
        output_hidden_states=False
    )
    model.to("cuda")
    with torch.no_grad():
        predictions = []
        for batch in loader:
            test_input_ids, test_masks = batch
            outputs = model(
                test_input_ids.to("cuda"),
                test_masks.to("cuda"),
            )
            predictions.append(outputs.logits.argmax(-1).cpu())
        df_test["label"] = torch.cat(predictions, dim=0).numpy()
        df_test["label"] = df_test["label"].map(
            {1: 'ai_answer', 0: 'hu_answer'})
        df_test[["line_id", "label"]].to_csv(
            f"{WORK_FOLDER}/data/submission.csv", sep=",", index=False)


In [ ]:
torch.cuda.empty_cache()